In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

PROCESSED_PATH = "C://Users//Rachit//OneDrive//Documents//CreditPathAI//data//processed"
DB_PATH = "C://Users//Rachit//OneDrive//Documents//CreditPathAI//data//processed//creditpath.db"


In [2]:
# loading the data set from "Processed_data folder":
# loaded all cleaned files
files = [f for f in os.listdir(PROCESSED_PATH) if f.endswith("_clean.csv")]

data = {}

for f in files:
    path = os.path.join(PROCESSED_PATH, f)
    df = pd.read_csv(path)
    data[f.replace("_clean.csv", "")] = df

data.keys()


dict_keys(['HC_application_train', 'HC_bureau_balance', 'HC_bureau', 'HC_credit_card_balance', 'HC_installments_payments', 'HC_POS_CASH_balance', 'HC_previous_application', 'HC_sample_submission', 'HomeCredit_columns_description'])

In [3]:
# Extracted the main application table
app = data["HC_application_train"].copy()

# Renamed key column for safety
app.rename(columns={"sk_id_curr": "SK_ID_CURR"}, inplace=True)

app.shape


(307511, 122)

In [7]:
# Quick check on table feilds:
bureau_merged.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'credit_active', 'credit_currency',
       'days_credit', 'credit_day_overdue', 'days_credit_enddate',
       'days_enddate_fact', 'amt_credit_max_overdue', 'cnt_credit_prolong',
       'amt_credit_sum', 'amt_credit_sum_debt', 'amt_credit_sum_limit',
       'amt_credit_sum_overdue', 'credit_type', 'days_credit_update',
       'amt_annuity', 'BB_MONTH_BAL_MEAN', 'BB_STATUS_COUNT'],
      dtype='object')

In [6]:
# Extracting tables
bureau = data["HC_bureau"].copy()
bureau_balance = data["HC_bureau_balance"].copy()

# Renaming key columns
bureau.rename(columns={"sk_id_curr": "SK_ID_CURR", "sk_id_bureau": "SK_ID_BUREAU"}, inplace=True)
bureau_balance.rename(columns={"sk_id_bureau": "SK_ID_BUREAU"}, inplace=True)

# --- Step 1: Aggregate bureau_balance at bureau level ---
bb_agg = bureau_balance.groupby("SK_ID_BUREAU").agg({
    "months_balance": "mean",
    "status": "nunique"
}).reset_index()

bb_agg.columns = ["SK_ID_BUREAU", "BB_MONTH_BAL_MEAN", "BB_STATUS_COUNT"]

# Merging this small aggregated table with bureau
bureau_merged = bureau.merge(bb_agg, on="SK_ID_BUREAU", how="left")

# --- Step 2: Aggregate at SK_ID_CURR (customer) level ---

bureau_final = bureau_merged.groupby("SK_ID_CURR").agg({
    "credit_active": "nunique",
    "credit_type": "nunique",
    "amt_credit_sum": "sum",
    "amt_credit_sum_debt": "sum",
    "amt_credit_sum_limit": "sum",
    "amt_credit_sum_overdue": "sum",
    "BB_MONTH_BAL_MEAN": "mean",
    "BB_STATUS_COUNT": "mean"
}).reset_index()

bureau_final.head()


,SK_ID_CURR,credit_active,credit_type,amt_credit_sum,amt_credit_sum_debt,amt_credit_sum_limit,amt_credit_sum_overdue,BB_MONTH_BAL_MEAN,BB_STATUS_COUNT
0,100001,2,1,1453365.000,596686.5,0.000,0.0,-11.785714,2.428571
1,100002,2,2,865055.565,245781.0,31988.565,0.0,-21.875000,3.250000
2,100003,2,2,1017400.500,0.0,810000.000,0.0,NaN,NaN
3,100004,1,1,189037.800,0.0,0.000,0.0,NaN,NaN
4,100005,2,2,657126.000,568408.5,0.000,0.0,-3.000000,2.000000


In [8]:
bureau_balance = data["HC_bureau_balance"].copy()
print("Columns in bureau_balance:")
bureau_balance.columns


Columns in bureau_balance:


Index(['sk_id_bureau', 'months_balance', 'status'], dtype='object')

In [18]:
# Loding main application data:

In [11]:
# Checking Files:
import os
os.listdir(PROCESSED_PATH)

['creditpath.db',
 'HC_application_train_clean.csv',
 'HC_bureau_balance_clean.csv',
 'HC_bureau_clean.csv',
 'HC_credit_card_balance_clean.csv',
 'HC_installments_payments_clean.csv',
 'HC_POS_CASH_balance_clean.csv',
 'HC_previous_application_clean.csv',
 'HC_sample_submission_clean.csv',
 'HomeCredit_columns_description_clean.csv']

In [ ]:
# Fixing error(Loding app):

app = pd.read_csv(
    os.path.join(PROCESSED_PATH, "HC_application_train_clean.csv")
)

print('Shape:',app.shape)

Shape:/n (307511, 122)


In [19]:
app.rename(columns={"sk_id_curr": "SK_ID_CURR"}, inplace=True)

In [ ]:
# Merging ....“app_fe created”

app_fe = app.merge(
    bureau_final,
    on="SK_ID_CURR",
    how="left"
)

app_fe.shape


(307511, 130)

In [22]:
# Saving the "Feature Engineered Dataset":

output_path = os.path.join(PROCESSED_PATH, "final_features.csv")
app_fe.to_csv(output_path, index=False)

output_path

'C://Users//Rachit//OneDrive//Documents//CreditPathAI//data//processed\\final_features.csv'